# MITx 6.00.1x: Problem Set 4  
Author: Leo Robinovitch

## Problem Set Summary: Play the Game

In [ ]:
from ps4b import *

if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)

## Introduction:
In this problem set, you'll implement two versions of a wordgame!

Don't be intimidated by the length of this problem set. There is a lot of reading, but it can be done with a reasonable amount of thinking and coding. It'll be helpful if you start this problem set a few days before it is due!

Let's begin by describing the 6.00 wordgame: This game is a lot like Scrabble or Words With Friends, if you've played those. Letters are dealt to players, who then construct one or more words out of their letters. Each valid word receives a score, based on the length of the word and the letters in that word.

The rules of the game are as follows:

Dealing
A player is dealt a hand of n letters chosen at random (assume n=7 for now).

The player arranges the hand into as many words as they want out of the letters, using each letter at most once.

Some letters may remain unused (these won't be scored).

Scoring
The score for the hand is the sum of the scores for each word formed.

The score for a word is the sum of the points for letters in the word, multiplied by the length of the word, plus 50 points if all n letters are used on the first word created.

Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is worth 3, D is worth 2, E is worth 1, and so on. We have defined the dictionary SCRABBLE_LETTER_VALUES that maps each lowercase letter to its Scrabble letter value.

For example, 'weed' would be worth 32 points ((4+1+1+2) for the four letters, then multiply by len('weed') to get (4+1+1+2)*4 = 32). Be sure to check that the hand actually has 1 'w', 2 'e's, and 1 'd' before scoring the word!

As another example, if n=7 and you make the word 'waybill' on the first try, it would be worth 155 points (the base score for 'waybill' is (4+1+4+3+1+1+1)*7=105, plus an additional 50 point bonus for using all n letters).

## Problem 1: Word Scores

In [ ]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    pointSum = 0
    score = 0
    
    for letter in word:
        pointSum += SCRABBLE_LETTER_VALUES[letter.lower()]
    
    score = pointSum * len(word)
    
    if len(word) == n:
        score += 50
    
    return score

## Problem 2: Dealing with Hands

In [ ]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    handCopy = hand.copy()
    for letter in word:
        handCopy[letter] = handCopy.get(letter, 1) - 1
    
    return handCopy

## Problem 3: Valid Words

In [ ]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    if word not in wordList:
        return False
    
    for letter in word:
        if hand.get(letter, 0) < word.count(letter):
            return False
        
    return True

## Problem 4: Hand Length

In [ ]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    lenHand = 0
    
    for key in hand.keys():
        lenHand += hand[key]
    
    return lenHand

## Problem 5: Playing a Hand

In [ ]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # Keep track of the total score
    totalScore = 0
    
    # As long as there are still letters left in the hand:
    while calculateHandlen(hand) > 0:
        # Display the hand
        print('Current Hand:', end=' ')
        displayHand(hand)
        # Ask user for input
        word = input('Enter word, or a "." to indicate that you are finished: ')
        # If the input is a single period:
        if word == '.':
            # End the game (break out of the loop)
            break
            
        # Otherwise (the input is not a single period):
        else:
            # If the word is not valid:
            if not isValidWord(word, hand, wordList):
                # Reject invalid word (print a message followed by a blank line)
                print('Invalid word, please try again.', '\n')
            # Otherwise (the word is valid):
            else:
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line
                totalScore += getWordScore(word, n)
                print(word + ' earned ' + str(getWordScore(word, n)) + ' points. Total: ' + str(totalScore) + ' points', '\n')
                # Update the hand 
                hand = updateHand(hand, word)

    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    if word == '.':
        print('Goodbye! Total score: ' + str(totalScore) + ' points.')
    else:
        print('Run out of letters. Total score: ' + str(totalScore) + ' points.') 

## Problem 6: Playing a Game

In [ ]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1
    """
    # Initialize variables
    currentHand = dict()
    
    # Start play-game loop
    while True:
        # Get user input on what to play
        while True:
            entry = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
            
            if entry in ['n', 'r', 'e']:
                break    
            else:
                print('Invalid command.')
        
        # New random hand
        if entry == 'n':
            currentHand = dealHand(HAND_SIZE)
            playHand(currentHand, wordList, HAND_SIZE)
    
        # Replay hand
        elif entry == 'r':
            if not currentHand:
                print('You have not played a hand yet. Please play a new hand first!')
            else:
                playHand(currentHand, wordList, HAND_SIZE)
    
        # Exit game:
        else:
            break

## Problem 7: You and Your Computer

In [ ]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
          But if no hand was played, output "You have not played a hand yet. 
          Please play a new hand first!"
        
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    # Initialize variables
    currentHand = dict()
    
    # Start play-game loop
    while True:
        # Get user input on what to play
        while True:
            playStyle = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
            
            if playStyle in ['n', 'r', 'e']:
                if playStyle == 'r' and not currentHand:
                    print('You have not played a hand yet. Please play a new hand first!')
                else:
                    break    
            else:
                print('Invalid command.')
        
        # Specify user or computer player
        while True and playStyle != 'e':
            player = input('Enter u to have yourself play, c to have the computer play: ')
            
            if player in ['u', 'c']:
                break    
            else:
                print('Invalid command.')
                
        # New random hand
        if playStyle == 'n':
            currentHand = dealHand(HAND_SIZE)
            
            if player == 'u':
                playHand(currentHand, wordList, HAND_SIZE)
            else:
                compPlayHand(currentHand, wordList, HAND_SIZE)
                
        # Replay hand
        elif playStyle == 'r':
            if player == 'u':
                playHand(currentHand, wordList, HAND_SIZE)
            else:
                compPlayHand(currentHand, wordList, HAND_SIZE)
                    
        # Exit game:
        else:
            break